In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import grad
import torch.nn.functional as F
import pandas as pd
import torch.nn.utils.parametrize as parametrize
from torch.nn.functional import normalize
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import r2_score, mean_squared_error

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [3]:
start = 1
data = pd.read_csv("data-B90102-30m.csv")
room = data["room_temp"]
airflow = data["airflow_current"]
supply = data["supply_temp"]

In [ ]:
delta =  np.array(room[start:-1]) - np.array(room[start-1:-2])
delta_supply = np.array(supply[start+1:]) - np.array(supply[start:-1])
delta_air = np.array(airflow[start+1:]) - np.array(airflow[start:-1])

temp_current = np.array(room[start:-1]).reshape(-1,1)
temp = np.array(room[start:-1]).reshape(-1,1)
output = np.array(room[start+1:]).reshape(-1,1)
air = np.array(airflow[start+1:]).reshape(-1,1)
supply = np.array(supply[start+1:]).reshape(-1,1)

air_origin = torch.tensor(air).to(device).reshape(-1,1).float()
supply_origin = torch.tensor(supply).to(device).reshape(-1,1).float()
temp_origin = torch.tensor(temp).to(device).reshape(-1,1).float()
output_origin = torch.tensor(output).to(device).reshape(-1,1).float()
temp_current = torch.tensor(temp_current).to(device).reshape(-1,1).float()

delta = torch.tensor(delta).reshape(-1,1).float().to(device)/(max(temp_origin) - min(temp_origin))
delta_supply = torch.tensor(delta_supply).reshape(-1,1).float().to(device)/(max(supply_origin) - min(supply_origin))
delta_air = torch.tensor(delta_air).reshape(-1,1).float().to(device)/(max(air_origin)- min(air_origin))

delta = torch.concat((delta, delta_supply, delta_air), dim =1)

In [ ]:
air = (air_origin - min(air_origin))/(max(air_origin)- min(air_origin))
air = air.requires_grad_(True)
supply = (supply_origin - min(supply_origin))/(max(supply_origin) - min(supply_origin))
supply = supply.requires_grad_(True)
temp = (temp_origin - min(temp_origin))/(max(temp_origin) - min(temp_origin))
temp = temp.requires_grad_(True)
output = (output - min(output))/(max(output) - min(output))
output = torch.tensor(output).reshape(-1,1).float().to(device)
temp_current = (temp_current - min(temp_origin))/(max(temp_origin) - min(temp_origin))

In [6]:
def data_train(N: int, N_end: int, temp_current: torch.Tensor, temp: torch.Tensor, supply: torch.Tensor, air: torch.Tensor, output: torch.Tensor, delta: torch.Tensor):
    temp_current_train = temp_current[N:N_end]
    temp_train = temp[N:N_end]
    supply_train = supply[N:N_end]
    air_train = air[N:N_end]
    output_train = output[N:N_end]
    # Delta delta
    delta_train = delta[N:N_end].requires_grad_(True)
    return temp_current_train, temp_train, supply_train, air_train, output_train, delta_train

def data_test(N: int, end: int, temp_current: torch.Tensor, temp: torch.Tensor, supply: torch.Tensor, air: torch.Tensor, output: torch.Tensor, delta: torch.Tensor):
    temp_current_test = temp_current[N:end]
    temp_test = temp[N:end]
    supply_test = supply[N:end]
    air_test = air[N:end]
    output_test = output[N:end]
    delta_test = delta[N:end]
    return temp_current_test, temp_test, supply_test, air_test, output_test, delta_test

def data_val(N: int, end: int, temp_current: torch.Tensor, temp: torch.Tensor, supply: torch.Tensor, air: torch.Tensor, output: torch.Tensor, delta: torch.Tensor):
    temp_current_val = temp_current[N:end]
    temp_val = temp[N:end]
    supply_val = supply[N:end]
    air_val = air[N:end]
    output_val = output[N:end]
    delta_val = delta[N:end]
    return temp_current_val, temp_val, supply_val, air_val, output_val, delta_val

In [ ]:
class ExponentialParameterization(nn.Module):
    def forward(self, X):
        return torch.exp(X)

class Max(nn.Module):
    def __init__(self):
        super(Max, self).__init__()

    def forward(self, x):
        return torch.max(x, dim=-1, keepdim=True)[0]

class Min(nn.Module):
    def __init__(self):
        super(Min, self).__init__()

    def forward(self, x):
        return torch.min(x, dim=-1)[0]

class CustomNet(nn.Module):
    def __init__(self):
        super(CustomNet, self).__init__()
        self.fc1_1 = nn.Linear(1, 4)  
        self.fc1_2 = nn.Linear(1, 4)
        self.fc1_3 = nn.Linear(1, 4)
        self.fc2 = nn.ModuleList([Max() for _ in range(2)])  
        self.fc3 = Min()  
        parametrize.register_parametrization(self.fc1_2, "weight", ExponentialParameterization())
    
    def forward(self, x, y, z):
        x_1 = self.fc1_1(x)  
        y_1 = self.fc1_2(y)
        z_1 = self.fc1_3(z)
        x = x_1 + y_1 + z_1
        split_size = x.size(1) // 2
        x_splits = [x[:, i*split_size:(i+1)*split_size] for i in range(2)]
        x = torch.cat([fc(split) for fc, split in zip(self.fc2, x_splits)], dim=-1)
        output = self.fc3(x).reshape(-1, 1)
        return output

net = CustomNet().to(device)


In [ ]:
epochs = 25000
patience = 1000
threshold = 1e-15
decay_factor = 1.0  
decay_step_size = 1000  
N_start, N_end = 360, 540
Ns_test, Ne_test = 950, 1050
Ns_val, Ne_val = 800, 900

temp_current_train, temp_train, supply_train, air_train, output_train, delta_train = data_train(N_start, N_end, temp_current, temp, supply, air, output, delta)
temp_current_test, temp_test, supply_test, air_test, output_test, delta_test = data_test(Ns_test, Ne_test, temp_current, temp, supply, air, output, delta)
temp_current_val, temp_val, supply_val, air_val, output_val, delta_val = data_val(Ns_val, Ne_val, temp_current, temp, supply, air, output, delta)

In [ ]:
learning = np.arange(0.001, 0.1, 0.002)
num_test = 1
current_mse_score = 0
mse = []
mse.append(10.0)
prev_loss = 10
for lr in learning:
    net = CustomNet().to(device)
    optimizer = optim.Adam(net.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2000, gamma=0.9)  # Adjust step_size and gamma as needed
    criterion = nn.MSELoss()
    for epoch in range(epochs):
        optimizer.zero_grad()
        output = net(temp_train, supply_train, air_train)
        t_next = output
        loss = criterion(t_next, output_train)
        loss.backward()
        optimizer.step()
        scheduler.step()  
        
        if epoch % 5000 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.9f}')
            
        if abs(prev_loss - loss.item()) < threshold:
            no_improvement_counter += 1
        else:
            no_improvement_counter = 0  
                
        prev_loss = loss.item()   
        if no_improvement_counter >= patience:
            print("Early stopping due to no significant improvement.")
            break
        if loss.item() < 0.00000063711:
            print("Loss function too small")
            break
    
    t_next = net(temp_val, supply_val, air_val)
    current_mse_score = mean_squared_error(output_val.to("cpu").detach().numpy(), t_next.to("cpu").detach().numpy())
    if current_mse_score < min(mse):
        torch.save(net.state_dict(), f"model_Min_Max_5_min.pth")
        mse.append(current_mse_score)
        print(current_mse_score)

Epoch [1/25000], Loss: 1.118266106
Epoch [5001/25000], Loss: 0.000911587
Epoch [10001/25000], Loss: 0.000366998
Epoch [15001/25000], Loss: 0.000246891
Epoch [20001/25000], Loss: 0.000238011
0.0002765938
Epoch [1/25000], Loss: 1.238411546
Epoch [5001/25000], Loss: 0.001082048
Epoch [10001/25000], Loss: 0.000323510
Epoch [15001/25000], Loss: 0.000238821
Epoch [20001/25000], Loss: 0.000238065
Epoch [1/25000], Loss: 0.166883275
Epoch [5001/25000], Loss: 0.000276335
Epoch [10001/25000], Loss: 0.000267752
Early stopping due to no significant improvement.
Epoch [1/25000], Loss: 0.275520980
Epoch [5001/25000], Loss: 0.000267438
Epoch [10001/25000], Loss: 0.000265864
Epoch [15001/25000], Loss: 0.000265864
Epoch [20001/25000], Loss: 0.000265864
Epoch [1/25000], Loss: 0.548583090
Epoch [5001/25000], Loss: 0.000265936
Epoch [10001/25000], Loss: 0.000269856
Epoch [15001/25000], Loss: 0.000265864
Epoch [20001/25000], Loss: 0.000279061
Epoch [1/25000], Loss: 0.477411836
Epoch [5001/25000], Loss: 0.00

In [ ]:
num_predict=7
r2_loop = []
mse_loop = []
loaded_model = CustomNet().to(device)
model_path = f"model_Min_Max_5_min.pth"
loaded_model.load_state_dict(torch.load(model_path))

loaded_model.eval()

t_next = loaded_model(temp_test[0:-num_predict], supply_test[0:-num_predict], air_test[0:-num_predict])
r2 = r2_score(output_test[0:-num_predict].to("cpu").detach().numpy(), t_next.to("cpu").detach().numpy())
mse = mean_squared_error(output_test[0:-num_predict].to("cpu").detach().numpy(), t_next.to("cpu").detach().numpy())
r2_loop.append(r2)
mse_loop.append(mse)
for i in list(range(1, num_predict)):
    t_next = loaded_model(t_next, supply_test[i:i-num_predict], air_test[i:i-num_predict])
    r2 = r2_score(output_test[i:i-num_predict].to("cpu").detach().numpy(), t_next.to("cpu").detach().numpy())
    mse = mean_squared_error(output_test[i:i-num_predict].to("cpu").detach().numpy(), t_next.to("cpu").detach().numpy())
    r2_loop.append(r2)
    mse_loop.append(mse)
r2_loop

[0.9425655397929681,
 0.8330258120007011,
 0.694461471469566,
 0.5431712326914875,
 0.3991854542129072,
 0.25755559329718103,
 0.13180816373670023]

In [72]:
np.sqrt(mse_loop).flatten().reshape(-1, 1)*7

array([[0.21817884],
       [0.36737227],
       [0.492254  ],
       [0.59620917],
       [0.6776514 ],
       [0.7470846 ],
       [0.80182767]], dtype=float32)